 
## Synthesis Runtime: This generates the synthetic data from a model provided to it

### Install Rockfish

In [ ]:
%%capture
%pip install -U 'rockfish[labs]==0.23.0' -f 'https://docs142.rockfish.ai/packages/index.html'

In [1]:
import rockfish as rf
import rockfish.actions as ra

import pickle

### Connect to the Rockfish Platform

❗❗ Replace API_KEY.

In [2]:
API_KEY = 'insert your API key here'

conn = rf.Connection.remote('https://api.rockfish.ai', API_KEY)

 
### Provide query params to the model_store search to get appropriate models as response

In [3]:
async for model in conn.list_models(labels={'usage': 'production'}):
    print(model)

Model(id='6a841e23-60da-11ef-b0af-b21fd42e2041', labels={'customer': 'hbo', 'month': 'june', 'workflow_id': '51AoIAxZw3sNM99H2s83wD'}, create_time=datetime.datetime(2024, 8, 22, 23, 1, 7, tzinfo=datetime.timezone.utc), size_bytes=26864128)


 
### Select a model from the list of queried models and fetch it from remote

In [4]:
model = await rf.Model.from_id(conn, 'model id here of the filtered model after querying')
print(model)

In [6]:
with open('generate_actions.pickle', 'rb') as f:
    generate_actions = pickle.load(f)

 
### Provide the model and the synthesis config to a workflow to generate a synthetic dataset as the output

In [7]:
builder = rf.WorkflowBuilder()
builder.add(model)
builder.add(*generate_actions, parents=[model], alias='gen')
builder.add(ra.DatasetSave(name='syn_data'), parents=['gen'])
workflow = await builder.start(conn)
print(f'Workflow ID: {workflow.id()}')

Workflow ID: 7azJsUA2r6fovNks6hF93O


In [8]:
syn_data = await workflow.datasets().concat(conn)

In [9]:
syn_data.to_pandas()

,sessionStartTimeMs,pageStartTimeMs,lifeSessionSessionDurationMs,lifeSessionPageLoadDurationMs,lifeSessionUserActiveTimeMs,lifeSessionNetworkRequestFailureDurationMs,lifeSessionNetworkRequestFailureCount,lifeSessionEventCount,lifeSessionUserEventCount,intvSessionDurationMs,...,asn,connType,netSpeed,browser,lifeSessionAppCrashCount,lifeSessionPageLoadSuccessCount,lifeSessionPageLoadAttemptCount,intvPageLoadSuccessCount,intvNetworkRequestFailureCount,session_key
0,2024-04-26 05:13:32.883,9751,2973442,1526,3270148,2000,1,150,12,12878,...,1136,15,Cellular,Chrome;Chrome 123.0.0.0,0,6,7,0,1,0.0
1,2024-04-26 04:11:49.457,13790,8978073,4109,119458,2127,0,-8,143,52064,...,50266,14,Cable/DSL,Native App;Native App,0,8,1,1,1,1.0
2,2024-04-26 07:10:24.584,14159,8665729,4109,-125857,2208,0,5,143,55803,...,50266,14,Cable/DSL,Native App;Native App,0,8,1,1,1,1.0
3,2024-04-26 10:07:55.914,14145,8581321,4109,-134053,2268,0,12,143,57064,...,50266,14,Cable/DSL,Native App;Native App,0,8,1,1,1,1.0
4,2024-04-26 13:07:30.336,14344,8334301,4109,-279604,2326,0,12,143,59130,...,50266,14,Cable/DSL,Native App;Native App,0,8,5,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10436,2024-05-14 08:13:02.236,4934,4988734,59,97064,2120,1,199,15,33786,...,31615,16,Cable/DSL,Chrome;Chrome 123.0.0.0,0,2,5,0,6,181.0
10437,2024-05-14 11:01:53.791,4951,4976269,56,94949,2119,1,198,15,33606,...,31615,16,Cable/DSL,Chrome;Chrome 123.0.0.0,0,2,5,0,6,181.0
10438,2024-05-14 13:50:44.880,4883,4994239,58,104530,2133,1,199,15,33431,...,31615,16,Cable/DSL,Chrome;Chrome 123.0.0.0,0,2,5,0,6,181.0
10439,2024-05-14 16:39:32.423,5024,4824629,58,70594,2120,1,198,15,33202,...,31615,16,Cable/DSL,Chrome;Chrome 123.0.0.0,0,2,5,0,6,181.0
